In [ ]:
from lib import geoSearch
import json

In [ ]:
GS = geoSearch.GeoSearch("elasticsearch-geo",9200)
from shapely.geometry import mapping, shape

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch-geo', 'port': 9200}])
from elasticsearch.client import IndicesClient
ic = IndicesClient(es)
def analyzedText(text):
    results = ic.analyze(index="montevideo", body=
    {
      "text": text, 
        "analyzer":"search_analyzer"
    })
    return ' '.join([token['token'] for token in results['tokens']])

In [ ]:
text = u"bv artigas av italia"
analyzedText(text)

In [ ]:
matches = GS.complete_search(text,50)
for nombre, match in matches[:4]:
    if match:
        shape_1 = shape(match['objects'][0]['geometry'])
        print ('✔️ MATCH: {}\t/\t{}'.format(match['score'],nombre))
print('\n')

In [ ]:
import pandas as pd
test_set = pd.read_csv('test_complete.csv', index_col=0, encoding='utf-8')

In [ ]:
test_set

In [ ]:
for index, row in test_set.iterrows():
    print(row.texto)
    print(row['mejorMatch'], row['scoreMejorMatch'])
    matches = GS.complete_search(row.texto,100)
    for nombre, match in matches[:5]:
        if match:
            print ('✔️ MATCH: {}\t/\t{}'.format(nombre, match['score'],match['type']))
    print('\n')

In [ ]:
GS.test_kit('test_complete.csv',1,1)